Goal: Check if bigger dataset

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.multioutput import RegressorChain
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline
import seaborn as sns
import pandas as pd
import warnings

In [ ]:
recordsUrl = '../data/rawRecords_150_8388608.csv'

dataset = pd.read_csv(recordsUrl)

In [ ]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
x_train=train_dataset['NumberN'].to_numpy().reshape(-1, 1)
y_train=train_dataset[['SmallerPrime','BiggerPrime']]

In [ ]:
# scale
warnings.filterwarnings("error", message=".*check_inverse*.",category=UserWarning, append=False)

xScaler = FunctionTransformer(func=np.log1p, inverse_func=np.expm1, validate=True, check_inverse=True)
xScaler.fit(x_train)
transformedRawX = xScaler.transform(x_train)
yScaler = FunctionTransformer(func=np.log1p, inverse_func=np.expm1, validate=True, check_inverse=True)
yScaler.fit(y_train)
transformedRawY = yScaler.transform(y_train)

Train

In [ ]:
regressor = MLPRegressor()
regressor.fit(transformedRawX, transformedRawY)

In [ ]:
print('regressor.score(x_test, y_test)=' + str(regressor.score))

plt.plot(regressor.loss_curve_)
plt.title("Loss Curve", fontsize=14)
plt.xlabel('Iterations')
plt.ylabel('Cost')
plt.show()

In [ ]:
x_test=test_dataset['NumberN'].to_numpy().reshape(-1, 1)
y_test=test_dataset[['SmallerPrime','BiggerPrime']]

xTestScaler = FunctionTransformer(func=np.log1p, inverse_func=np.expm1, validate=True, check_inverse=True)
xTestScaler.fit(x_test)
x_test_transformed = xScaler.transform(x_test)
yTestScaler = FunctionTransformer(func=np.log1p, inverse_func=np.expm1, validate=True, check_inverse=True)
yTestScaler.fit(y_test)
y_test_transformed = yScaler.transform(y_test)

In [ ]:
print('regressor.score(x_test_transformed, y_test_transformed)=' + str(regressor.score(x_test_transformed, y_test_transformed)))

x_test_zero = x_test_transformed[-1:]
y_test_zero = y_test_transformed[-1:]
y_pred = regressor.predict(x_test_zero)
y_pred_zero = y_pred[0]
print(f'x_test_zero={str(x_test_zero)} y_pred_zero={str(y_pred_zero)} y_test_zero={str(y_test_zero)}')

y_pred = regressor.predict(x_test_transformed)
plt.scatter(x_test_transformed, y_test_transformed[:, 0], color="black")
plt.scatter(x_test_transformed, y_pred[:, 0], color="blue")
plt.show()


In [ ]:
x_test_rawx = int(x_test[-1:][0])
y_test_rawyS = int(y_test.iloc[-1]['SmallerPrime'])
y_pred_invTransS = yTestScaler.inverse_transform(y_pred[-1:])
print(y_pred_invTransS)
y_pred_invTransS_zero = int(y_pred_invTransS[0][0])
pred_diff = y_test_rawyS - y_pred_invTransS_zero
print(f'x_test_zero={str(x_test_rawx)} pred_diff={str(pred_diff)} y_pred_invTransS_zero={str(y_pred_invTransS_zero)} y_test_rawyS={str(y_test_rawyS)}')